This script includes the CNN based network to train, quantize and compile model used for gesture 
recognition from Electromyograpgy( EMG ) and Electrnecephalography (EEG). The EMG data was acquired using the commercial Myo-Armband, while the EEG data was acquired using the OPenBCI ultracortex "Mark IV" 

(https://docs.openbci.com/AddOns/Headwear/MarkIV/#:~:text=The%20Ultracortex%20is%20an%20open,love%20to%20hear%20your%20feedback.). Myo-Armband has a sampling frequency of 200Hz with 8 channels placed on the forearm while the ULtracortex has a sampling frequency of 250Hz. 

Below are the steps.
1. Importing and Pre-processing --> This step also includes notching the input signal at 60Hz and windowig with over. Because the DPU engine require three channel input, the shape of the data is expanded to fit DPU requirement. 

2. Training --> After setting the hyperparamters,the architecture was defined and the trained using tensorflow 2.* Therafter the accuracy of the model was visualized to confirm performance. 

3. Quantization --> Post training quantization was implemented in this script using the Vitis-AI tool. The quantizer reduces the precision of 32 bit floating point to 8-bit fixed point.

4. Compling --> The subgraph needed to be complied to produce .xmodel format after would later to deployed to the Xilix FPGA (Ultra-96 v2) as an overlay in Pynq.

In [ ]:
#!pip install mlxtend
#!pip install tensorflow-addons

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow_model_optimization.quantization.keras import vitis_quantize
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.constraints import max_norm
from tensorflow_model_optimization.quantization.keras import vitis_quantize
from tensorflow.keras.utils import plot_model

from bmis_hybrid_utils import *

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics
from scipy import signal

 1. Data Importation

In [ ]:
subject = 33

no_gesture = 7

emg_fs = 200
eeg_fs = 250

notch_freq = 60.0
quality_factor = 30.0


eeg_fc = 10.0
eeg_fh = 35.0

emg_fc = 10.0
emg_fh = 99.0

order = 5
window_time = 200
overlap = 60
no_channel = 8

In [ ]:
emg_data, eeg_data, label = get_data_subject_specific(subject, no_gesture, emg_fs, eeg_fs,
                                                      notch_freq, quality_factor, emg_fc, emg_fh, 
                                                      eeg_fc, eeg_fh, order,
                                                      window_time, overlap, no_channel)

print(f'Total EMG data is {emg_data.shape} EEG data is {eeg_data.shape} Label {label.shape}')

In [ ]:
np.unique(label)

In [ ]:
# EMG data split check this well
X_emg_train, y_train, X_emg_test, y_test = spilt_data(emg_data, label, ratio=0.1)
print('EMG Training Set is{} Test Set {}'.format(X_emg_train.shape, X_emg_test.shape))

In [ ]:
# EEG data split
X_eeg_train, _, X_eeg_test, _ = spilt_data(eeg_data, label, ratio=0.1)
print('EEG Training Set is{} Test Set {}'.format(X_eeg_train.shape, X_eeg_test.shape))

 2. Data Preparation 

In [ ]:
# Downsampling EEG
X_eeg_train = downsample_data(data=X_eeg_train, nfs=emg_fs, window_time=window_time)
X_eeg_test = downsample_data(data=X_eeg_test, nfs=emg_fs, window_time=window_time)

In [ ]:
print(f'Resampled EEG training data is: {X_eeg_train.shape}')
print(f'Resampled EEG test data is: {X_eeg_test.shape}')

In [ ]:
# Expanding the input feature shape for EMG
X_emg_train = np.expand_dims(X_emg_train, axis=3)
X_emg_test = np.expand_dims(X_emg_test, axis=3)
print('Expanded Dimension are {}'.format(X_emg_train.shape))
input_size = X_emg_train.shape[1:]
print(input_size)

In [ ]:
# Expanding the input feature shape for EEG
X_eeg_train = np.expand_dims(X_eeg_train, axis=3)
X_eeg_test = np.expand_dims(X_eeg_test, axis=3)
print('Expanded Dimension are {}'.format(X_eeg_train.shape))
input_size = X_emg_train.shape[1:]
print(input_size)

In [ ]:
gesture_list = ['LDG', 'MRDG', 'TFSG', 'PPG', 'PG', 'Cut', 'Rest']

3. Importing Pre-trained model for transfer learning

In [ ]:
emg_model_path = '/workspace/sEMG/BMIS_EMG_DATA/full_models'
emg_subject = 'EMG-NET-' + str(subject) + '.h5'
base_model_path = os.path.join(emg_model_path, emg_subject)
base_model = tf.keras.models.load_model(base_model_path)
base_model.summary()

In [ ]:
feature_extractor = tf.keras.Model(inputs=base_model.inputs, outputs=[layer.output for layer in base_model.layers[0:5]])
feature_extractor.summary()

In [ ]:
eeg_input=base_model.inputs
intermediate=[layer.output for layer in base_model.layers[1:5]]

In [ ]:
def hybrid_net(input_size, eeg_input, intermediate):
    
    
    emg_input = tf.keras.Input(shape=input_size,  name="emg")
    x = tf.keras.layers.Conv2D(32, 3, activation="relu", input_shape=input_size)(emg_input)
    x = tf.keras.layers.Conv2D(32, 3, activation="relu")(x)
    emg_output = tf.keras.layers.MaxPool2D((2,2))(x)
    
    
    x = intermediate[-1]
    eeg_output = x
  
    
    concat = tf.keras.layers.concatenate([emg_output, eeg_output])
    
    down_stream_layer = tf.keras.layers.Flatten()(concat)
    down_stream_layer = tf.keras.layers.Dropout(0.5)(down_stream_layer)
    down_stream_layer = tf.keras.layers.Dense(100, activation='relu')(down_stream_layer)
    down_stream_layer = tf.keras.layers.Dense(25, activation='relu')(down_stream_layer)
    output = tf.keras.layers.Dense(7, activation='softmax')(down_stream_layer)
    
    hybrid_net = tf.keras.Model([emg_input, eeg_input], output, name='BMIS_Hybrid_Net')
    
    return hybrid_net

In [ ]:
model = hybrid_net(input_size, eeg_input, intermediate)
model.summary()

In [ ]:
X_train = {'emg':X_emg_train, 'eeg':X_eeg_train}
X_test = {'emg':X_emg_test, 'eeg':X_eeg_test}

In [ ]:
lr = 0.0001
#opt = tf.keras.optimizers.Adam(learning_rate=lr)
opt = 'adam'
ls = 'sparse_categorical_crossentropy'
mtr = 'accuracy'
n_batches = 32
n_epochs = 50
#callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=1)
model.compile(optimizer=opt, loss=ls, metrics=mtr)
num_folds = 3
fold_no = 1
kfold = KFold(n_splits=num_folds, shuffle=False)
accuracy_per_fold = []
loss_per_fold = []

In [ ]:
for train, test in kfold.split(X_train['emg'], y_train):
    
    model = hybrid_net(input_size, eeg_input, intermediate)
    model.compile(optimizer=opt, loss=ls, metrics=mtr)
    
    print('---------------------------------------------------')
    print(f'Training for fold {fold_no} -------')
    
    history = model.fit((X_train['emg'][train],X_train['eeg'][train]),
                        y_train[train], batch_size=n_batches, epochs= n_epochs, verbose=1)
    
    scores = model.evaluate((X_train['emg'][test],X_train['eeg'][test]), y_train[test], verbose=0)
    print(f'Score for fold  {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    accuracy_per_fold.append(scores[1] *100)
    loss_per_fold.append(scores[0])
          
    fold_no = fold_no + 1

In [ ]:
print("Average Score per fold ")

for i in range(0, len(accuracy_per_fold)):
    print('-----------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {accuracy_per_fold[i]}%')
print('-----------------------------------------------')
print('Average Metrics for all folds: ')
print(f'> Accuracy: {np.mean(accuracy_per_fold)} (+- {np.std(accuracy_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('-----------------------------------------------')

In [ ]:
saved_model = "single-Subject-HYBRID-NET.h5" 
model.save(saved_model)

In [ ]:
calibration_sample = 1000
calibration_data = (X_train['emg'][calibration_sample],X_train['eeg'][calibration_sample])

In [ ]:
saved_float32_model = tf.keras.models.load_model(saved_model)
ptq_quantizer = vitis_quantize.VitisQuantizer(saved_float32_model)
ptq_quantized_model = ptq_quantizer.quantize_model(calib_dataset=X_train)

In [ ]:
# Evalauting Post training quantization

ptq_quantized_model.compile(loss=ls, metrics=mtr)
ptq_quantized_model.evaluate(X_test, y_test)

In [ ]:
quantized_model = "single-Subject-quantized-HYBRID-NET.h5"
ptq_quantized_model.save(quantized_model)

In [ ]:
!vai_c_tensorflow2 \
    --model ../ptq_models/ptq-Hybrid-Net-33.h5 \
    --arch ./arch_ultra96.json \
    --output_dir ../inference_models \
    --net_name subject_33_model 

In [ ]:
num_folds = 3
fold_no = 1
kfold_2 = KFold(n_splits=num_folds, shuffle=False)
accuracy_per_fold_2 = []
loss_per_fold_2 = []
n_epochs = 37

In [ ]:
for train, test in kfold_2.split(X_train['emg'], y_train):
    
    qat_model = hybrid_net()
    quantizer = vitis_quantize.VitisQuantizer(qat_model, quantize_strategy='8bit_tqt')
    qat_model_set = quantizer.get_qat_model(init_quant=True, calib_dataset=X_train)
    
    qat_model.compile(optimizer= opt, loss=ls, metrics=mtr)
    
    print('---------------------------------------------------')
    print(f'Quantization Aware Training for fold {fold_no} -------')
    
    history = qat_model.fit((X_train['emg'][train],X_train['eeg'][train]), y_train[train], batch_size=n_batches, epochs= n_epochs, verbose=1)
    
    scores = qat_model.evaluate((X_train['emg'][test],X_train['eeg'][test]), y_train[test], verbose=0)
    print(f'Score for fold  {fold_no}: {qat_model.metrics_names[0]} of {scores[0]}; {qat_model.metrics_names[1]} of {scores[1]*100}%')
    accuracy_per_fold_2.append(scores[1] *100)
    loss_per_fold_2.append(scores[0])
          
    fold_no = fold_no + 1

In [ ]:
print("Average Score per fold ")

for i in range(0, len(accuracy_per_fold_2)):
    print('-----------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold_2[i]} - Accuracy: {accuracy_per_fold_2[i]}%')
print('-----------------------------------------------')
print('Average Metrics for all folds: ')
print(f'> Accuracy: {np.mean(accuracy_per_fold_2)} (+- {np.std(accuracy_per_fold_2)})')
print(f'> Loss: {np.mean(loss_per_fold_2)}')
print('-----------------------------------------------')

Visualize Model

In [ ]:
plot_model(model, to_file='hybrid_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
model = hybrid_net(input_size, eeg_input, intermediate)

In [ ]:
model.summary()

In [ ]:
input_layer_names = []
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.InputLayer):
        input_layer_names.append(layer.name)

print(input_layer_names)

In [ ]:
input_size = (8, 40, 1)
subject = 13

#include path to the 32bit model example below
emg_model_path = '/workspace/sEMG/BMIS_EMG_DATA/full_models'
emg_subject = 'EMG-NET-' + str(subject) + '.h5'
base_model_path = os.path.join(emg_model_path, emg_subject)
base_model = tf.keras.models.load_model(base_model_path)



eeg_input=base_model.inputs
intermediate=[layer.output for layer in base_model.layers[1:5]]



def hybrid_net(input_size, eeg_input, intermediate):
    
    
    emg_input = tf.keras.Input(shape=input_size,  name="emg")
    x = tf.keras.layers.Conv2D(32, 3, activation="relu", input_shape=input_size)(emg_input)
    x = tf.keras.layers.Conv2D(32, 3, activation="relu")(x)
    emg_output = tf.keras.layers.MaxPool2D((2,2))(x)
    
    
    x = intermediate[-1]
    eeg_output = x
  
    
    concat = tf.keras.layers.concatenate([emg_output, eeg_output])
    
    down_stream_layer = tf.keras.layers.Flatten()(concat)
    down_stream_layer = tf.keras.layers.Dropout(0.5)(down_stream_layer)
    down_stream_layer = tf.keras.layers.Dense(100, activation='relu')(down_stream_layer)
    down_stream_layer = tf.keras.layers.Dense(25, activation='relu')(down_stream_layer)
    output = tf.keras.layers.Dense(7, activation='softmax')(down_stream_layer)
    
    hybrid_net = tf.keras.Model([emg_input, eeg_input], output, name='BMIS_Hybrid_Net')
    
    return hybrid_net



def run_hybrid_net_experiment(start_subject=13, stop_subject=14, split_ratio=0.2, no_gesture=7, 
                              emg_fs=200, eeg_fs=250, notch_freq=60.0, quality_factor=30.0, 
                              eeg_fc=10.0, eeg_fh=35.0, emg_fc=10.0, emg_fh=99.0,order=5, window_time=200,
                              overlap=60, no_channel=8, opt='adam', ls='sparse_categorical_crossentropy', 
                              mtr='accuracy', n_batches=16, n_epochs=1):
    
    
    result = pd.DataFrame({
        'Subject': [0],
        'Validation_result':[0.0],
        'qat_Validation_result':[0.0],
        'Test_result_32_bit':[0.0],
        'Test_result_8_bit_ptq':[0.0],
        'Test_result_8_bit_qat':[0.0],
        
        'precision_full':[0.0],
        'recall_full':[0.0],
        'f1_score_full':[0.0],

        'precision_ptq':[0.0],
        'recall_ptq':[0.0],
        'f1_score_ptq':[0.0],
        
        'precision_qat':[0.0],
        'recall_qat':[0.0],
        'f1_score_qat':[0.0]
        })
    
    
    emg_data, eeg_data, label = get_data_subject_specific(start_subject, no_gesture, emg_fs, eeg_fs,
                                                      notch_freq, quality_factor, emg_fc, emg_fh, 
                                                      eeg_fc, eeg_fh, order,
                                                      window_time, overlap, no_channel)

    #print(f'Total EMG data is {emg_data.shape} EEG data is {eeg_data.shape} Label {label.shape}')
    
    
    # EMG data split check this well
    X_emg_train, y_train, X_emg_test, y_test = spilt_data(emg_data, label, ratio=split_ratio)
    # EEG data split
    X_eeg_train, _, X_eeg_test, _ = spilt_data(eeg_data, label, ratio=split_ratio)
    
    #X_eeg_train = X_eeg_train[:len(X_emg_train)]
    #X_eeg_test = X_eeg_train[:len(X_emg_test)]
    
    
    # Downsampling EEG
    X_eeg_train = downsample_data(data=X_eeg_train, nfs=emg_fs, window_time=window_time)
    X_eeg_test = downsample_data(data=X_eeg_test, nfs=emg_fs, window_time=window_time)
    
    
    # Expanding the input feature shape for EMG
    X_emg_train = np.expand_dims(X_emg_train, axis=3)
    X_emg_test = np.expand_dims(X_emg_test, axis=3)
    
    # Expanding the input feature shape for EEG
    X_eeg_train = np.expand_dims(X_eeg_train, axis=3)
    X_eeg_test = np.expand_dims(X_eeg_test, axis=3)
    input_size = X_emg_train.shape[1:]
    
    X_train = {'emg':X_emg_train, 'eeg':X_eeg_train}
    X_test = {'emg':X_emg_test, 'eeg':X_eeg_test}
    
    for subject in range(start_subject, (stop_subject+1)):
        
        fold_no = 1
        num_folds = 3
        accuracy_per_fold = []
        loss_per_fold = []
        qat_accuracy_per_fold = []
        qat_loss_per_fold = []
        kfold = KFold(n_splits=num_folds, shuffle=False)
        input_layer_names = []
        
        
        emg_model_path = '/workspace/sEMG/BMIS_EMG_DATA/full_models'
        emg_subject = 'EMG-NET-' + str(subject) + '.h5'
        base_model_path = os.path.join(emg_model_path, emg_subject)
        base_model = tf.keras.models.load_model(base_model_path)



        eeg_input=base_model.inputs
        intermediate=[layer.output for layer in base_model.layers[1:5]]
        
        
        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
        print(f'Training and Evaluation for subject {subject}')
        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
        
        
            
        emg_data, eeg_data, label = get_data_subject_specific(subject, no_gesture, emg_fs, eeg_fs,
                                                          notch_freq, quality_factor, emg_fc, emg_fh, 
                                                          eeg_fc, eeg_fh, order,
                                                          window_time, overlap, no_channel)

        #print(f'Total EMG data is {emg_data.shape} EEG data is {eeg_data.shape} Label {label.shape}')


        # EMG data split check this well
        X_emg_train, y_train, X_emg_test, y_test = spilt_data(emg_data, label, ratio=split_ratio)
        # EEG data split
        X_eeg_train, _, X_eeg_test, _ = spilt_data(eeg_data, label, ratio=split_ratio)

        #X_eeg_train = X_eeg_train[:len(X_emg_train)]
        #X_eeg_test = X_eeg_train[:len(X_emg_test)]


        # Downsampling EEG
        X_eeg_train = downsample_data(data=X_eeg_train, nfs=emg_fs, window_time=window_time)
        X_eeg_test = downsample_data(data=X_eeg_test, nfs=emg_fs, window_time=window_time)


        # Expanding the input feature shape for EMG
        X_emg_train = np.expand_dims(X_emg_train, axis=3)
        X_emg_test = np.expand_dims(X_emg_test, axis=3)

        # Expanding the input feature shape for EEG
        X_eeg_train = np.expand_dims(X_eeg_train, axis=3)
        X_eeg_test = np.expand_dims(X_eeg_test, axis=3)
        input_size = X_emg_train.shape[1:]
        
        model_check_name = hybrid_net(input_size, eeg_input, intermediate)
        for layer in model_check_name.layers:
            if isinstance(layer, tf.keras.layers.InputLayer):
                input_layer_names.append(layer.name)
        
        
        X_train = {'emg':X_emg_train, 'eeg':X_eeg_train}
        X_test = {'emg':X_emg_test, input_layer_names[0]:X_eeg_test}
        
        
        
 
        
        calibration_dataset = {'emg':X_emg_train[:1000], input_layer_names[0]:X_eeg_train[:1000]} # Calibration data needed to quantize the model
        print(f'Input shape to the Hybrid-Net Model is: {input_size}')
        
        
        for train, test in kfold.split(X_train['emg'], y_train):
    
            model = hybrid_net(input_size, eeg_input, intermediate)
            model.compile(optimizer=opt, loss=ls, metrics=mtr)
            checkpoint_path = os.path.join('../checkpoint/full', str(subject))

            if not os.path.exists(checkpoint_path):
                os.makedirs(checkpoint_path)

            early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10)
            checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                        filepath = checkpoint_path, save_best_only=True,
                                        monitor='accuracy', vebrose=1)
    
            print('---------------------------------------------------')
            print(f'Training Unquantized Model for fold {fold_no} -------')
    
            history = model.fit((X_train['emg'][train],X_train['eeg'][train]), y_train[train], 
                                callbacks=[early_stop, checkpoint_callback], 
                                batch_size=n_batches, epochs= n_epochs, verbose=1)
        
    
            scores = model.evaluate((X_train['emg'][test],X_train['eeg'][test]), y_train[test], verbose=0)
            print(f'Score for fold  {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
            accuracy_per_fold.append(scores[1] *100)
            loss_per_fold.append(scores[0])
          
            
            
            print('######################################################')
            print('Quantization Aware Training')
            
            qat_checkpoint_path = os.path.join('../checkpoint/qat', str(subject))

            if not os.path.exists(qat_checkpoint_path):
                os.makedirs(qat_checkpoint_path)
     

            qat_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                        filepath = qat_checkpoint_path, save_best_only=True, monitor='accuracy', vebrose=1)
            
            qat_model = hybrid_net(input_size, eeg_input, intermediate)
            quantizer = vitis_quantize.VitisQuantizer(qat_model, quantize_strategy='8bit_tqt')
            qat_model_set = quantizer.get_qat_model(init_quant=True, calib_dataset=calibration_dataset)

            qat_model.compile(optimizer= opt, loss=ls, metrics=mtr)

            print('---------------------------------------------------')
            print(f'Quantization Aware Training for fold {fold_no} -------')

            qat_history = qat_model.fit((X_train['emg'][train],X_train['eeg'][train]), y_train[train], 
                                        callbacks=[early_stop, qat_checkpoint_callback], 
                                        batch_size=n_batches, epochs= n_epochs, verbose=1)

            qat_scores = qat_model.evaluate((X_train['emg'][test],X_train['eeg'][test]), y_train[test], verbose=0)
            print(f'QAT Score for fold  {fold_no}: {qat_model.metrics_names[0]} of {qat_scores[0]}; {qat_model.metrics_names[1]} of {qat_scores[1]*100}%')
            qat_accuracy_per_fold.append(qat_scores[1] *100)
            qat_loss_per_fold.append(scores[0])

            fold_no = fold_no + 1     

        
        print("Average Full Bit Validation Score per fold ")

        for i in range(0, len(accuracy_per_fold)):
            print('-----------------------------------------------')
            print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {accuracy_per_fold[i]}%')
        print('-----------------------------------------------')
        print('Average Metrics for all folds: ')
        print(f'> Accuracy: {np.mean(accuracy_per_fold)} (+- {np.std(accuracy_per_fold)})')
        print(f'> Loss: {np.mean(loss_per_fold)}')
        print('-----------------------------------------------')
        print('############################################################')
        print(f'Training Ended for subject {subject}')
        print('############################################################')
        
        validation_result = np.mean(accuracy_per_fold)
        qat_validation_result = np.mean(qat_accuracy_per_fold)
        
        ######### Load the best model from checkpoint. ################
        model = tf.keras.models.load_model(checkpoint_path)
        
        ############# Evaluating 32-bit Model ##########################
        print('############################################################')
        print(f'Evaluating Unquantized Model')
        print('############################################################')
        uqt_test_loss, uqt_test_accuracy = model.evaluate(X_test, y_test)
        full_test_accuracy = uqt_test_accuracy * 100
        print(f'Accuracy of the Unquantized_model {full_test_accuracy}%')
        
        
        y_predict = model.predict(X_test)
        y_predict = np.argmax(y_predict, axis=-1)
   
        precision = precision_score(y_test, y_predict, average='weighted')
        recall = recall_score(y_test, y_predict, average='weighted')
        f1_score_full = 2 * (precision * recall) / (precision + recall)


        
        print(f'Precision of the Unquantized model {precision}')
        print(f'Recall of the Unquantized model {recall}')
        print(f'F1_score of the Unquantized model {f1_score_full}')
    
        
        ############ Save the full model as .h5. This will be used for quantization #####
        saved_model = '../full_models/Hybrid-Net-' + str(subject) + '.h5'
        model.save(saved_model)
        
        
        
        ############### Post-Training Quantization ########################
        saved_float32_model = tf.keras.models.load_model(saved_model)
        ptq_quantizer = vitis_quantize.VitisQuantizer(saved_float32_model)
        ptq_model = ptq_quantizer.quantize_model(calib_dataset=calibration_dataset)
        
        
        ############# Evalauting PTQ Model #######################################
        print('############################################################')
        print(f'Evaluating PTQ Model')
        print('############################################################')

        ptq_model.compile(loss=ls, metrics=mtr)
        ptq_loss, ptq_test_accuracy = ptq_model.evaluate(X_test, y_test)
        ptq_test_accuracy = ptq_test_accuracy * 100
        print(f'Accuracy of the PTQ model {ptq_test_accuracy}%')
        
        
            
        ptq_y_predict = ptq_model.predict(X_test)
        ptq_y_predict = np.argmax(ptq_y_predict, axis=-1)
        ptq_precision = precision_score(y_test, ptq_y_predict, average='weighted')
        ptq_recall = recall_score(y_test, ptq_y_predict, average='weighted')
        ptq_f1_score = 2 * (ptq_precision * ptq_recall) / (ptq_precision + ptq_recall)
        
        
        print(f'Precision of the PTQ model {ptq_precision}')
        print(f'Recall of the PTQ model {ptq_recall}')
        print(f'F1_score of the PTQ model {ptq_f1_score}')

        # Saving PTQ model; Can be complied for depolyment #######  
        ptq_quantized_model = '../ptq_models/ptq-Hybrid-Net-' + str(subject) + '.h5'
        ptq_model.save(ptq_quantized_model)
        
        
        ############# Evalauting QAT Model ###################################
        
        ## Load Best QAT Model
        qat_model = tf.keras.models.load_model(qat_checkpoint_path)
        
        ## ######### Evaluate QAT Model###############################################
        print('############################################################')
        print(f'Evaluating QAT Model')
        print('############################################################')
        
        qat_test_loss, qat_test_accuracy = qat_model.evaluate(X_test, y_test)
        qat_test_accuracy = qat_test_accuracy * 100
        print(f'Accuracy of the QAT model {qat_test_accuracy}%')
        
        
        qat_y_predict = qat_model.predict(X_test)
        qat_y_predict = np.argmax(qat_y_predict, axis=-1)
        qat_precision = precision_score(y_test, qat_y_predict, average='weighted')
        qat_recall = recall_score(y_test, qat_y_predict, average='weighted')
        qat_f1_score = 2 * (qat_precision * qat_recall) / (qat_precision + qat_recall)
        
        
        print(f'Precision of the QAT model {qat_precision}')
        print(f'Recall of the QAT model {qat_recall}')
        print(f'F1_score of the QAT model {qat_f1_score}')
        
        # Saving QAT model; Can be complied for depolyment #######  
        qat_quantized_model = '../qat_models/qat-Hybrid-Net-' + str(subject) + '.h5'
        qat_model.save(qat_quantized_model)
        
        
        result.at[subject-1, 'Subject'] = subject
        result.at[subject-1, 'Validation_result'] = validation_result
        result.at[subject-1, 'qat_Validation_result'] = qat_validation_result
        result.at[subject-1, 'Test_result_32_bit'] = full_test_accuracy
        result.at[subject-1, 'Test_result_8_bit_ptq'] = ptq_test_accuracy
        result.at[subject-1, 'Test_result_8_bit_qat'] = qat_test_accuracy
        
        result.at[subject-1, 'precision_full'] = precision 
        result.at[subject-1, 'recall_full'] = recall
        result.at[subject-1, 'f1_score_full'] = f1_score_full
   
        result.at[subject-1, 'precision_ptq'] = ptq_precision 
        result.at[subject-1, 'recall_ptq'] = ptq_recall
        result.at[subject-1, 'f1_score_ptq'] = ptq_f1_score
        
        result.at[subject-1, 'precision_qat'] = qat_precision 
        result.at[subject-1, 'recall_qat'] = qat_recall
        result.at[subject-1, 'f1_score_qat'] = qat_f1_score

        save_path = str(start_subject)+'_to_'+str(stop_subject)+'_new_Hybrid-Net.csv'
        save_path = os.path.join('../results', save_path)
        result.to_csv(save_path, index=False)

In [ ]:
run_hybrid_net_experiment(start_subject=1, stop_subject=33, split_ratio=0.1, no_gesture=7, 
                          emg_fs=200, eeg_fs=250, notch_freq=60.0, quality_factor=30.0, 
                          eeg_fc=10.0, eeg_fh=35.0, emg_fc=10.0, emg_fh=99.0,order=5, window_time=200,
                          overlap=60, no_channel=8, opt='adam', ls='sparse_categorical_crossentropy', 
                          mtr='accuracy', n_batches=16, n_epochs=50)